In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from featuresv2 import features, targets, metadata
import json
import requests
%matplotlib inline

## Version 2 Model

In [3]:
data = pd.read_csv('outputs/model_training_data_v2.csv')
data_2023 = data[(data['round.year']!= 2023)]

In [3]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [4]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [11]:
features

['kicks|HOME',
 'handballs|HOME',
 'disposals|HOME',
 'marks|HOME',
 'bounces|HOME',
 'tackles|HOME',
 'contestedPossessions|HOME',
 'uncontestedPossessions|HOME',
 'totalPossessions|HOME',
 'inside50s|HOME',
 'marksInside50|HOME',
 'contestedMarks|HOME',
 'hitouts|HOME',
 'onePercenters|HOME',
 'clangers|HOME',
 'freesFor|HOME',
 'freesAgainst|HOME',
 'rebound50s|HOME',
 'goalAssists|HOME',
 'turnovers|HOME',
 'intercepts|HOME',
 'tacklesInside50|HOME',
 'shotsAtGoal|HOME',
 'scoreInvolvements|HOME',
 'metresGained|HOME',
 'clearances.centreClearances|HOME',
 'clearances.stoppageClearances|HOME',
 'clearances.totalClearances|HOME',
 'extendedStats.effectiveKicks|HOME',
 'extendedStats.effectiveDisposals|HOME',
 'extendedStats.marksOnLead|HOME',
 'extendedStats.interceptMarks|HOME',
 'extendedStats.hitoutsToAdvantage|HOME',
 'extendedStats.groundBallGets|HOME',
 'extendedStats.f50GroundBallGets|HOME',
 'extendedStats.scoreLaunches|HOME',
 'extendedStats.pressureActs|HOME',
 'extendedSt

In [4]:
def generate_test_train_split(data, features, target, test_size=0.3):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [5]:
def tpot_pipeline_v2(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], data_['y_train'])
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [6]:
def return_stats_input(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
#     if 'team.name' not in model_inputs:
#         model_inputs.append('team.name')
        
    match_ups = data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)]
    weights = range(1,len(match_ups)+1)

    stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))
    
    return pd.DataFrame(dict(zip(model_inputs,stats[0])),index=[0])

In [7]:
def generate_margin_tear_off(model, round_matchups, team_stats, model_inputs):
    print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        predict = model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
        if predict < 0:
            winner = tup[1]
        else:
            winner = tup[0]
        print("------------------------------------------------")
        print("Home: " + tup[0])
        print("Away: " + tup[1])
        print("Winner: " + winner + "\t       " + "Margin: " + str(abs(round(predict))))

In [8]:
def generate_score_tear_off(home_model, away_model, round_matchups, team_stats, model_inputs):
   # print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    preds = []
    for tup in round_matchups:
        try:
            h_predict = home_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            a_predict = away_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            if h_predict < a_predict:
                winner = tup[1]
                margin = round(a_predict) - round(h_predict)
            else:
                winner = tup[0]
                margin = round(h_predict) - round(a_predict)
            paddingh = 25 - len("Home: " + tup[0])
            paddinga = 25 - len("Away: " + tup[1])
            print("------------------------------------------------")
            print("Home: " + tup[0] + " "*paddingh + str(abs(round(h_predict))))
            print("Away: " + tup[1] + " "*paddinga + str(abs(round(a_predict))))
            print("Winner: " + winner + " by "+ str(margin) + " points")
            print("Total Score: " + str(round(h_predict) + round(a_predict)))
            preds.append((h_predict, a_predict, margin))
        except:
            print("No prior game")
            preds.append((0, 0, 0))
    return preds

In [18]:
home_model = tpot_pipeline_v2(data, features, targets[0], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9530894193915331

Generation 2 - Current best internal CV score: 0.9530894193915331

Generation 3 - Current best internal CV score: 0.9530894193915331

Generation 4 - Current best internal CV score: 0.9533335594323157

Generation 5 - Current best internal CV score: 0.9536697183831494

Best pipeline: RidgeCV(MaxAbsScaler(RidgeCV(input_matrix)))
0.9642324551772503


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


In [19]:
away_model = tpot_pipeline_v2(data, features, targets[1], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9573357800268001

Generation 2 - Current best internal CV score: 0.9580553758732337

Generation 3 - Current best internal CV score: 0.9581026542152735

Generation 4 - Current best internal CV score: 0.9581081161149058

Generation 5 - Current best internal CV score: 0.9581081161149058

Best pipeline: LinearSVR(MaxAbsScaler(input_matrix), C=20.0, dual=False, epsilon=0.1, loss=squared_epsilon_insensitive, tol=1e-05)
0.9587851353717809


In [ ]:
#margin_model = tpot_pipeline_v2(data, features, targets[2], size=10)

In [ ]:
#generate_score_tear_off(home_model, away_model, round1, data, features)

In [ ]:
# print(return_stats_inputv3(data, home_team, away_team, features))
#generate_margin_tear_off(margin_model, round1, data, features)
# data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)][features].head()

In [20]:
fix_data_2024 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2024').content)

fix_data_2023 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2023').content)

fix_data_2022 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2022').content)

fixtures2024 = pd.DataFrame(fix_data_2024)
games2024 = list(zip(fixtures2024.HomeTeam, fixtures2024.AwayTeam))

fixtures2023 = pd.DataFrame(fix_data_2023)
games2023 = list(zip(fixtures2023.HomeTeam, fixtures2023.AwayTeam))

fixtures2022 = pd.DataFrame(fix_data_2022)
games2022 = list(zip(fixtures2022.HomeTeam, fixtures2022.AwayTeam))

predicted_season = generate_score_tear_off(home_model, away_model, games2024, data, features)

fixtures2024['predicted_home'] = [round(i[0]) for i in predicted_season]
fixtures2024['predicted_away'] = [round(i[1]) for i in predicted_season]

C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Sydney Swans       68
Away: Melbourne          70
Winner: Melbourne by 2 points
Total Score: 138
------------------------------------------------
Home: Brisbane Lions     107
Away: Carlton            70
Winner: Brisbane Lions by 37 points
Total Score: 177
------------------------------------------------
Home: Gold Coast Suns    69
Away: Richmond           98
Winner: Richmond by 29 points
Total Score: 167
------------------------------------------------
Home: GWS Giants         87
Away: Collingwood        75
Winner: GWS Giants by 12 points
Total Score: 162


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Carlton            83
Away: Richmond           92
Winner: Richmond by 9 points
Total Score: 175
------------------------------------------------
Home: Collingwood        62
Away: Sydney Swans       49
Winner: Collingwood by 13 points
Total Score: 111
------------------------------------------------
Home: Essendon           96
Away: Hawthorn           85
Winner: Essendon by 11 points
Total Score: 181


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


...

------------------------------------------------
Home: Western Bulldogs   95
Away: North Melbourne    76
Winner: Western Bulldogs by 19 points
Total Score: 171
------------------------------------------------
Home: Port Adelaide      93
Away: Adelaide Crows     70
Winner: Port Adelaide by 23 points
Total Score: 163
------------------------------------------------
Home: West Coast Eagles  59
Away: Carlton            109
Winner: Carlton by 50 points
Total Score: 168
------------------------------------------------
Home: Brisbane Lions     87
Away: Essendon           71
Winner: Brisbane Lions by 16 points
Total Score: 158


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Carlton            68
Away: St Kilda           78
Winner: St Kilda by 10 points
Total Score: 146
------------------------------------------------
Home: Geelong Cats       127
Away: West Coast Eagles  63
Winner: Geelong Cats by 64 points
Total Score: 190
------------------------------------------------
Home: Hawthorn           80
Away: North Melbourne    66
Winner: Hawthorn by 14 points
Total Score: 146
------------------------------------------------
Home: Melbourne          77
Away: Collingwood        75
Winner: Melbourne by 2 points
Total Score: 152


C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\SeanW\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(


------------------------------------------------
Home: Richmond           53
Away: Gold Coast Suns    55
Winner: Gold Coast Suns by 2 points
Total Score: 108
------------------------------------------------
Home: Sydney Swans       102
Away: Adelaide Crows     86
Winner: Sydney Swans by 16 points
Total Score: 188
------------------------------------------------
Home: Western Bulldogs   65
Away: GWS Giants         65
Winner: Western Bulldogs by 0 points
Total Score: 130
------------------------------------------------
Home: Fremantle          72
Away: Port Adelaide      79
Winner: Port Adelaide by 7 points
Total Score: 151


In [23]:
pred_winner = []
winner = []
for i, row in fixtures2024.iterrows():

    if row[7] > row[8]:
        winner.append(row[4])
    elif row[8] > row[7]:
        winner.append(row[5])
    else:
        winner.append('Draw')


    if row[9] > row[10]:
        pred_winner.append(row[4])
    elif row[10] > row[9]:
        pred_winner.append(row[5])
    else:
        pred_winner.append('Draw')

fixtures2024['predicted_winner'] = pred_winner
fixtures2024['winner'] = winner
fixtures2024['predicted_margin'] = fixtures2024['predicted_home'] - fixtures2024['predicted_away']
fixtures2024['actual_margin'] = fixtures2024['HomeTeamScore'] - fixtures2024['AwayTeamScore']
fixtures2024['correct'] = [1 if row[11]==row[12] else 0 for i,row in fixtures2024.iterrows()]
# fixtures2022['bet'] = ["1-39" if abs(row[13]) < 40 else "40+" for i,row in fixtures2022.iterrows()]


fixtures2024.to_csv('outputs/predictions_and_results_2024_.csv', index=False)

# backtest_data = pd.read_csv('outputs/predictions_and_results_2022.csv')
# backtest_data.head()

In [20]:
fixtures2022.head()

,MatchNumber,RoundNumber,DateUtc,Location,HomeTeam,AwayTeam,Group,HomeTeamScore,AwayTeamScore,predicted_home,predicted_away
0,1,1,2022-03-16 08:10:00Z,MCG,Melbourne,Western Bulldogs,None,97,71,114,76
1,2,1,2022-03-17 08:25:00Z,MCG,Carlton,Richmond,None,101,76,63,94
2,3,1,2022-03-18 08:50:00Z,Marvel Stadium,St Kilda,Collingwood,None,85,102,0,0
3,4,1,2022-03-19 03:10:00Z,MCG,Geelong Cats,Essendon,None,138,72,96,49
4,5,1,2022-03-19 06:10:00Z,Accor Stadium,GWS Giants,Sydney Swans,None,92,112,77,91
